In [1]:
# Dependencies
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url_mars = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url_mars)

In [4]:
html_mars = browser.html

In [5]:
soup_mars = BeautifulSoup(html_mars, 'lxml')

In [6]:
news_title = soup_mars.find("div", class_='content_title').text
news_title

"Curiosity Tastes First Sample in 'Clay-Bearing Unit'"

In [7]:
news_paragraph = soup_mars.find("div", class_='article_teaser_body').text
news_paragraph

'This new region on Mars might reveal more about the role of water on Mount Sharp.'

In [8]:
#JPL Images
url_jpl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_jpl)
browser.is_text_present("FULL IMAGE")
browser.click_link_by_partial_text("FULL IMAGE")


In [9]:
html_jpl = browser.html

In [10]:
soup_jpl = BeautifulSoup(html_jpl, 'lxml')

In [11]:
 image_location = soup_jpl.find('article')['style'].replace('background-image: url(', '').replace(');', '')[1:-1]
image_location

'/spaceimages/images/wallpaper/PIA14934-1920x1200.jpg'

In [12]:
featured_image_url = "https://www.jpl.nasa.gov" + image_location
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14934-1920x1200.jpg'

In [13]:
#scrape mars weather from their tweeter account
url_mars_weather = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url_mars_weather)

In [14]:
html_mars_weather = browser.html

In [15]:
soup_mars_weather = BeautifulSoup(html_mars_weather, 'lxml')

In [16]:
mars_weather = soup_mars_weather.find('p', class_='tweet-text').text.replace('/n', '').split("pic")[0]
mars_weather

'InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF)\nwinds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph)\npressure at 7.30 hPa'

In [17]:
#scrape mars facts page using pandas
mars_facts_url = 'https://space-facts.com/mars/'

In [18]:
mars_tables = pd.read_html(mars_facts_url)
mars_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [19]:
df = mars_tables[0]
df.columns = ['title', 'value']

In [20]:
df.head()

,title,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [31]:
html_facts = df.to_html(index_names=False, index=False, justify='left')
html_facts.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: left;">      <th>title</th>      <th>value</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [23]:
# scrape USGA Astrogeology to get images of Mar's Hemispheres
mars_hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [24]:
browser.visit(mars_hemi_url)
html_mars_hemi = browser.html
soup_mars_hemi = BeautifulSoup(html_mars_hemi, 'lxml')

In [27]:
hemisphere_image_urls = []
base_url = "https://astrogeology.usgs.gov"
    
results = soup_mars_hemi.find_all("div", class_='item')
results

for result in results:
    image_url = result.find('a', class_='itemLink')['href']
    hemi_url = base_url + image_url
    browser.visit(hemi_url)
    hemi_html = browser.html
    hemi_soup = BeautifulSoup(hemi_html, 'lxml')
    title = hemi_soup.find('h2', class_='title').text
    image_loc = hemi_soup.find('a', target="_blank")["href"].strip()
    image_dict = {
        "title": title, "img_url": image_loc
    }
    hemisphere_image_urls.append(image_dict)
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
